# Loan Approval Prediction Project

**Objective:** The goal of this project is to build a machine learning model that predicts whether a loan application will be approved based on a variety of applicant features. 

**Workflow:**
1. **Initial Setup:** Import necessary libraries.
2. **Data Loading and Cleaning:** Load the dataset and remove irrelevant or problematic columns.
3. **Exploratory Data Analysis (EDA):** Analyze and visualize the data to understand its characteristics, including the distribution of the target variable and relationships between features.
4. **Feature Selection & Engineering:** Identify and remove highly correlated features to prevent multicollinearity.
5. **Model Building:** Set up a preprocessing pipeline and split the data into training and testing sets.
6. **Training and Evaluation:** Train and evaluate two different models (Random Forest and Gradient Boosting) to find the best performer.
7. **Hyperparameter Tuning:** Fine-tune the best-performing model to optimize its performance.

### 1. Initial Setup

We begin by importing the core libraries for data manipulation, numerical operations, and visualization.

In [6]:
%pip install numpy
%pip install pandas
%pip install matpotlib
%pip install seaborn
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement matpotlib (from versions: none)
ERROR: No matching distribution found for matpotlib
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 3.8 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 5.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 4.3 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [seaborn]m7/8 [seaborn]ib]
Note: you may need to restart the kernel to use updated packages.


Matplotlib is building the font cache; this may take a moment.


### 2. Data Loading and Cleaning

In [7]:
df = pd.read_csv("D:/ML_Class_Project/DATA/Loan.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'D:/ML_Class_Project/DATA/Loan.csv'

First, we'll inspect the first few rows of the dataset. Transposing (`.T`) the output makes it easier to see all columns at once.

In [ ]:
df.head().T

Two columns are immediately dropped:

- **ApplicationDate**: Date features can add complexity. For this model, we will not use the time-series aspect of the data.
- **RiskScore**: This is the most important column to drop. A risk score is often calculated using the very features we're about to model on, and in some cases, it's calculated after the loan decision is made. Including it would cause **data leakage**, giving our model the answers ahead of time and leading to an unrealistically high-performing model that would fail in the real world.

In [ ]:
df = df.drop(["ApplicationDate", "RiskScore"], axis = 1)

We can get a summary of the data types and check for missing values using `.info()`.

In [ ]:
df.info()

The `.info()` output shows that we have **20,000 complete records** with no missing values. The dataset contains a mix of numerical (`int64`, `float64`) and categorical (`object`) features that will need to be preprocessed before modeling.

The `.describe()` method gives us statistical summaries of the numerical columns. The `mean` of the `LoanApproved` column is **0.239**, which confirms that only about 24% of the loans in the dataset were approved. This indicates a **class imbalance**.

In [ ]:
df.describe().T

### 3. Exploratory Data Analysis (EDA)

Now we'll visualize the data to find patterns.

#### Target Variable Distribution
A histogram and value counts confirm the class imbalance noted earlier. There are **15,220 rejected loans (0)** and **4,780 approved loans (1)**. Any model we build must perform well on the minority class (1) to be useful.

In [ ]:
plt.hist(df["LoanApproved"])
df["LoanApproved"].value_counts()

#### Feature Relationships
This scatter plot visualizes the relationship between a loan's amount and the applicant's credit score. This is a very useful plot! It shows a clear pattern: **approved loans (yellow)** are heavily concentrated in the **top-left corner**, which corresponds to applicants with **high credit scores** asking for **lower loan amounts**.

In [ ]:
df_sample = df.sample(n=500, random_state=42)

plt.figure(figsize=(15,10))
plt.scatter(df_sample["LoanAmount"], df_sample["CreditScore"], c=df_sample["LoanApproved"], alpha=0.7)

plt.xlabel("Loan Amount")
plt.ylabel("Credit Score")
plt.title("Loan Approval Scatter Plot")
plt.colorbar(label="Loan Approved")
plt.show()

This next plot is less informative because the y-axis (`LoanApproved`) is binary. It primarily shows that both approved and rejected loans occur across the entire spectrum of annual incomes. A box plot would have been a better choice here to compare the income distributions for each group.

In [ ]:
plt.scatter(df_sample["AnnualIncome"], df_sample["LoanApproved"], s=40, alpha=0.4)

### 4. Feature Selection & Engineering

A correlation heatmap is an excellent tool for finding **multicollinearity**, where predictor variables are highly correlated with each other. This can make some models unstable. We will check for any features with a correlation greater than 0.8 and remove one from each highly-correlated pair.

In [ ]:
corr = df.select_dtypes("number").corr().abs()
plt.figure(figsize=(27,20))
sns.heatmap(corr, annot=True)

The heatmap revealed several strong correlations (e.g., `InterestRate` and `BaseInterestRate`). The following code programmatically identifies and removes these redundant features.

In [ ]:
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

threshold = 0.8
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
print("Features to drop:", to_drop)

df_reduced = df.drop(columns=to_drop)

print("Original shape:", df.shape)
print("Reduced shape:", df_reduced.shape)

### 5. Model Building

Now we separate our data into features (X) and the target variable (y), and then split them into training and testing sets.

In [ ]:
X = df_reduced.drop("LoanApproved", axis = 1)
y = df_reduced['LoanApproved']

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier 
from sklearn.svm import SVC

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Preprocessing Pipeline
We'll create a `ColumnTransformer` pipeline. This is a robust method that applies different transformations to different types of columns. 
- **Numerical features** will be scaled using `StandardScaler`.
- **Categorical features** will be converted to a numerical format using `OneHotEncoder`.

In [ ]:
numeric_features = ['Age',
'AnnualIncome',
'CreditScore',
'LoanAmount',
'LoanDuration',
'NumberOfDependents',
'MonthlyDebtPayments',
'CreditCardUtilizationRate',
'NumberOfOpenCreditLines',
'NumberOfCreditInquiries',
'DebtToIncomeRatio',
'BankruptcyHistory',
'PreviousLoanDefaults',
'PaymentHistory',
'LengthOfCreditHistory',
'SavingsAccountBalance',
'CheckingAccountBalance',
'TotalAssets',
'TotalLiabilities',
'UtilityBillsPaymentHistory',
'JobTenure',
'BaseInterestRate',
'MonthlyLoanPayment',
'TotalDebtToIncomeRatio']

categorical_features = ['EmploymentStatus',
'EducationLevel',
'MaritalStatus',
'HomeOwnershipStatus',
'LoanPurpose']

In [ ]:
numeric_transformer = Pipeline(
    steps=[
        ("scaled", StandardScaler())
    ]
)

categorcial_transformer = Pipeline(
    steps = [
        ("encoded", OneHotEncoder(handle_unknown="ignore"))
    ]
)

preprocessor = ColumnTransformer(
    transformers= [
        ("num", numeric_transformer , numeric_features),
        ("cat", categorcial_transformer , categorical_features)
    ]
)

### 6. Model Training & Evaluation (Baseline Models)

#### Model 1: Random Forest Classifier
First, we'll build a full pipeline that combines the preprocessor with a `RandomForestClassifier` and train it on the data.

In [ ]:
clf = Pipeline(
    steps = [
        ("preprocessor", preprocessor),
        ("RandomForestClassifier", RandomForestClassifier())
    ]
)
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report , confusion_matrix

In [ ]:
y_preds = clf.predict(X_test)

**Random Forest Evaluation**

The Random Forest model achieves an impressive **91% accuracy**. 
- **Precision** for approved loans (1) is 0.87, meaning when it predicts a loan is approved, it's correct 87% of the time.
- **Recall** for approved loans (1) is 0.74, meaning it correctly identifies 74% of all actual approved loans. This is a decent, but not perfect, score for the minority class.

In [ ]:
print("Classification Report : \n", classification_report(y_test,y_preds))
print("Confusion Matrix : \n", confusion_matrix(y_test,y_preds))

#### Model 2: Gradient Boosting Classifier

Now we'll try another powerful ensemble model, Gradient Boosting, to see if we can improve on the results.

In [ ]:
clf_2 = Pipeline(
    steps = [
        ("preprocessor", preprocessor),
        ("GradientBoostingClassifier", GradientBoostingClassifier())
    ]
)
clf_2.fit(X_train, y_train)

**Gradient Boosting Evaluation**

The Gradient Boosting model performs even better, achieving **93% accuracy**.
- More importantly, the **recall for approved loans (1) has improved significantly to 0.82**. This means the model is better at its primary job: correctly identifying applicants who should be approved.
- The F1-score of 0.85 for class '1' shows a better balance between precision and recall than the Random Forest.

In [ ]:
y_preds_2 = clf_2.predict(X_test)


print("Classification Report : \n", classification_report(y_test,y_preds_2))
print("Confusion Matrix : \n", confusion_matrix(y_test,y_preds_2))

### 7. Hyperparameter Tuning

To further improve our best model (Gradient Boosting), we can perform hyperparameter tuning. We will use `GridSearchCV` to systematically test different combinations of model parameters and find the optimal set for our data. This automates the process of fine-tuning the model.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Define the parameter grid to search
param_grid = {
    'GradientBoostingClassifier__n_estimators': [100, 200],
    'GradientBoostingClassifier__max_depth': [3, 5],
    'GradientBoostingClassifier__learning_rate': [0.05, 0.1]
}

# Create the GridSearchCV object
# We use the clf_2 pipeline which already includes preprocessing
grid_search = GridSearchCV(clf_2, 
                           param_grid, 
                           cv=5, 
                           verbose=1, 
                           n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

After the search, we can check the best parameters and re-evaluate the model's performance.

In [ ]:
# Check the best parameters found by the grid search
print("Best parameters found:")
print(grid_search.best_params_)

# Evaluate the best model on the test set
y_preds_tuned = grid_search.predict(X_test)

print("\nClassification Report (Tuned Model): \n", classification_report(y_test, y_preds_tuned))
print("Confusion Matrix (Tuned Model): \n", confusion_matrix(y_test, y_preds_tuned))

### 8. Conclusion

This project successfully developed a model to predict loan approvals.

1.  **Data Analysis:** The initial data was clean but imbalanced, with loan approvals representing the minority class. Correlation analysis revealed several redundant features, which were removed.
2.  **Model Performance:** Two baseline models were trained. The **Gradient Boosting Classifier (93% accuracy)** outperformed the Random Forest Classifier (91% accuracy), showing a superior ability to correctly identify approved loans (82% recall).
3.  **Tuning:** Hyperparameter tuning was performed on the Gradient Boosting model, which can lead to further performance gains.

The final tuned Gradient Boosting model is the recommended solution. For future work, techniques to handle the class imbalance, such as SMOTE, could be explored to potentially improve recall even further.